In [1]:
import torch
import torchvision
print(torch.__version__, torchvision.__version__)


2.7.0.dev20250127+cu124 0.22.0.dev20250127+cu124


In [1]:
from comfy_script.runtime import *

# load('http://127.0.0.1:8188/')
load()

# Nodes can only be imported after load()
from comfy_script.runtime.nodes import *

ComfyScript: Importing ComfyUI from /home/ubuntu/ComfyUI
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-01-28 23:21:38.894
** Platform: Linux
** Python version: 3.12.8 | packaged by conda-forge | (main, Dec  5 2024, 14:24:40) [GCC 13.3.0]
** Python executable: /home/ubuntu/ComfyUI/venv/bin/python
** ComfyUI Path: /home/ubuntu/ComfyUI
** ComfyUI Base Folder Path: /home/ubuntu/ComfyUI
** User directory: /home/ubuntu/ComfyUI/user
** ComfyUI-Manager config path: /home/ubuntu/ComfyUI/user/default/ComfyUI-Manager/config.ini
** Log path: /home/ubuntu/ComfyUI/user/comfyui.log



Prestartup times for custom nodes:
   2.0 seconds: /home/ubuntu/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Total VRAM 22716 MB, total RAM 15788 MB
pytorch version: 2.7.0.dev20250127+cu124
Set vram state to: NORMAL_VRAM
Device: cuda:0 NVIDIA A10G : cudaMallocAsync
Using pytorch attention
ComfyUI version: 0.3.12
[Prompt Server] web root: /home/ubuntu/ComfyUI/web
### Loading: ComfyUI-Manager (V3.9.5)
### ComfyUI Version: v0.3.12-26-g255edf22 | Released on '2025-01-27'
[ComfyUI-Manager] default cache updated: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/alter-list.json
[ComfyUI-Manager] default cache updated: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/model-list.json
[comfyui_controlnet_aux] | INFO -> Using ckpts path: /home/ubuntu/ComfyUI/custom_nodes/comfyui_controlnet_aux/ckpts
[comfyui_controlnet_aux] | INFO -> Using symlinks: False
[comfyui_controlnet_aux] | INFO -> Using ort providers: ['CUDAExecutionP

Ib Custom Nodes: Loaded
ComfyScript: Loaded
Nodes: 368



Import times for custom nodes:
   0.0 seconds: /home/ubuntu/ComfyUI/custom_nodes/websocket_image_save.py
   0.0 seconds: /home/ubuntu/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus
   0.1 seconds: /home/ubuntu/ComfyUI/custom_nodes/ComfyUI-Manager
   0.5 seconds: /home/ubuntu/ComfyUI/custom_nodes/ComfyScript
   0.9 seconds: /home/ubuntu/ComfyUI/custom_nodes/comfyui_controlnet_aux

Starting server

To see the GUI go to: http://127.0.0.1:8188


FETCH ComfyRegistry Data: 5/31
FETCH ComfyRegistry Data: 10/31
ComfyScript: __main__.__file__ is redirected to ComfyUI/main.py to keep compatibility with some nodes


...

[ComfyUI-Manager] default cache updated: https://api.comfy.org/nodes


 [DONE]


Using pytorch attention in VAE
Using pytorch attention in VAE
VAE load device: cuda:0, offload device: cpu, dtype: torch.bfloat16
Requested to load SDXLClipModel
loaded completely 9.5367431640625e+25 1560.802734375 True
CLIP/text encoder model load device: cuda:0, offload device: cpu, current: cuda:0, dtype: torch.float16
loaded diffusion model directly to GPU
Requested to load SDXL
loaded completely 9.5367431640625e+25 4897.0483474731445 True
Requested to load AutoencoderKL
loaded completely 2315.067802429199 159.55708122253418 True


model_path is /home/ubuntu/ComfyUI/custom_nodes/comfyui_controlnet_aux/ckpts/lllyasviel/Annotators/ZoeD_M12_N.pt


/home/ubuntu/ComfyUI/venv/lib/python3.12/site-packages/torch/functional.py:540: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4311.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
# with Workflow():
#     model, clip, vae = CheckpointLoaderSimple('sd_xl_base_1.0.safetensors')
#     conditioning = CLIPTextEncode('beautiful scenery nature glass bottle landscape, , purple galaxy bottle,', clip)
#     conditioning2 = CLIPTextEncode('text, watermark', clip)
#     latent = EmptyLatentImage(512, 512, 1)
#     latent = KSampler(model, 156680208700286, 20, 8, 'euler', 'normal', conditioning, conditioning2, latent, 1)
#     image = VAEDecode(latent, vae)
#     SaveImage(image, 'ComfyUI')

In [2]:
with Workflow():
    model, clip, vae = CheckpointLoaderSimple('sd_xl_base_1.0.safetensors')
    model, ipadapter = IPAdapterUnifiedLoader(model, 'STANDARD (medium strength)', None)
    image, _ = LoadImage('modern_office_ip.jpg')
    model = IPAdapterAdvanced(model, ipadapter, image, 1, 'style transfer', 'concat', 0, 1, 'V only', None, None, None)
    conditioning = CLIPTextEncode('A photorealistic image of a clean, modern, coworking office space that is well-lit.', clip)
    conditioning2 = CLIPTextEncode('text, watermark, low resolution, dark, old, people', clip)
    control_net = ControlNetLoader('SDXL/controlnet-canny-sdxl-1.0/diffusion_pytorch_model_V2.safetensors')
    image2, _ = LoadImage('workspace_sketch.jpg')
    image3 = CannyEdgePreprocessor(image2, 100, 200, 512)
    positive, negative = ControlNetApplyAdvanced(conditioning, conditioning2, control_net, image3, 1, 0, 1, None)
    control_net2 = ControlNetLoader('SDXL/depth-zoe-xl-v1.0-controlnet.safetensors')
    image4 = ZoeDepthMapPreprocessor(image2, 512)
    positive2, negative2 = ControlNetApplyAdvanced(positive, negative, control_net2, image4, 1, 0, 1, None)
    latent = VAEEncode(image3, vae)
    latent = KSampler(model, 63339159988543, 40, 6, 'ddim', 'normal', positive2, negative2, latent, 1)
    image5 = VAEDecode(latent, vae)
    PreviewImage(image5)

100%|##########| 1/1 [00:00<?, ?it/s]

100%|##########| 1/1 [00:00<?, ?it/s]